## Physics Teacher v2

Physics teacher now with a calculator tool




1. Preguntar a chatGPT si el formato es correcto (si se conforma con buenas prácticas) o puede mejorarse 
2. Explorar y entender el nuevo código, agregar tanto comentarios como explicación en Markdown
3. Codear Physics Teacher v3 que utilice las dos utilidades descritas abajo

In [27]:
# %% ----------------- imports & setup -----------------
import os, ast, operator as op
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in .env")

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
# We define the tool with the proper JSON schema
calc_tool = {
    "type": "function",
    "function": {
        "name": "calc",
        "description": "Evaluate an arithmetic expression and return the numeric result.",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "Pure arithmetic, e.g. '2 * pi * sqrt(0.75 / 9.81)'"
                }
            },
            "required": ["expression"]
        },
    },
}


In [29]:
_ALLOWED_OPS = {
    ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul,
    ast.Div: op.truediv, ast.Pow: op.pow, ast.Mod: op.mod,
    ast.USub: op.neg
}

def safe_eval(expr: str):
    """Evaluate a basic arithmetic expression safely."""
    def _eval(node):
        if isinstance(node, ast.Num): return node.n
        if isinstance(node, ast.UnaryOp): return _ALLOWED_OPS[type(node.op)](_eval(node.operand))
        if isinstance(node, ast.BinOp):
            return _ALLOWED_OPS[type(node.op)](_eval(node.left), _eval(node.right))
        raise ValueError("Unsupported expression")
    return _eval(ast.parse(expr, mode="eval").body)


In [30]:
system_message = (
    "You are a helpful, concise assistant for a high-school physics teacher. "
    "Answer physics questions clearly. "
    "If the user's request is not about physics, respond that you can only help with physics topics."
)



In [31]:
import json

def stream_chat(message: str, history: list, model_choice: str):
    # ---------- build conversation so far ----------
    base_msgs = [{"role": "system", "content": system_message}]
    for u, b in history:
        base_msgs += [{"role": "user", "content": u},
                      {"role": "assistant", "content": b}]
    base_msgs.append({"role": "user", "content": message})

    # ---------- 1️⃣ first, non-streaming request ----------
    first = client.chat.completions.create(
        model=model_choice,
        messages=base_msgs,
        tools=[calc_tool],
        stream=False          # blocking
    )
    choice   = first.choices[0]
    assistant_with_call = choice.message      # <-- keep this whole message

    # ---------- tool path ----------
    if choice.finish_reason == "tool_calls":
        call = assistant_with_call.tool_calls[0]
        args = json.loads(call.function.arguments)
        expr = args["expression"]
        print("🔧 calc called with:", expr)

        # run calculator safely
        try:
            result = safe_eval(expr)
        except Exception as e:
            result = f"Error: {e}"
        print("🔧 result:", result)

        # ---------- 2️⃣ send result + stream final reply ----------
        follow_up_msgs = (
            base_msgs
            + [assistant_with_call]           # 👈 REQUIRED
            + [{
                "role": "tool",
                "tool_call_id": call.id,
                "name": "calc",
                "content": str(result)
              }]
        )

        partial = ""
        for chunk in client.chat.completions.create(
            model=model_choice,
            messages=follow_up_msgs,
            stream=True
        ):
            partial += chunk.choices[0].delta.content or ""
            yield partial

    # ---------- no tool needed ----------
    else:
        partial = ""
        for chunk in client.chat.completions.create(
            model=model_choice,
            messages=base_msgs,
            stream=True
        ):
            partial += chunk.choices[0].delta.content or ""
            yield partial



In [32]:
# %% ----------------- Gradio UI -----------------
gr.ChatInterface(
    fn=stream_chat,
    chatbot=gr.Chatbot(label="Physics-Teacher Chatbot"),
    additional_inputs=[
        gr.Dropdown(
            choices=["gpt-4o", "gpt-4o-mini", "gpt-3.5-turbo"],
            value="gpt-4o",
            label="OpenAI model"
        )
    ],
    textbox=gr.Textbox(placeholder="Ask a physics question…"),
    title="🧑‍🏫 Physics Teacher with Calculator",
    theme="default"
).launch()

C:\Users\deluc\AppData\Local\Temp\ipykernel_32\1215131419.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(label="Physics-Teacher Chatbot"),
c:\Users\deluc\Documents\Agentic-Code\projects\agents\.venv\Lib\site-packages\gradio\chat_interface.py:321: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


🔧 calc called with: 5 / 1000
🔧 result: 0.005


C:\Users\deluc\AppData\Local\Temp\ipykernel_32\1232039094.py:10: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num): return node.n
C:\Users\deluc\AppData\Local\Temp\ipykernel_32\1232039094.py:10: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  if isinstance(node, ast.Num): return node.n


## Appendix - Using LLMs to create tools

Create JSON tool from natural language description  

Create a tool function from a JSON tool description